In [3]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [4]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


Eu tenho um arquivo secret.py com uma informação sigilosa. Neste caso é uma chave.
Eu vou chamar o conteudo dessa chave usando o código abaixo.


In [5]:
from secret import IEX_CLOUD_API_TOKEN

## Fazer a primeira chamada a API

A api que eu estou usando

A documentação da API que eu estou usando se encontra aqui: https://iexcloud.io/docs/api/


Eu vou buscar na API duas informações importantes:
O market capitalization para cada ação e o preço de cada ação.

Abaixo eu vou usar fstream para passar o simbolo para o endereço.

In [6]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url)
# print(data.status_code)
# Para eu converter o dado recebido na request em json eu só preciso
# Usar o comando abaixo.
data = requests.get(api_url).json()
print(data)


{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'OL S CGEMASGAAATE( NNLKBDRLQ/ETS)', 'calculationPrice': 'close', 'open': 133.89, 'openTime': 1629576151205, 'openSource': 'aioifflc', 'close': 131.08, 'closeTime': 1610815992966, 'closeSource': 'liaoicff', 'high': 137, 'highTime': 1675599821557, 'highSource': 'piamduredine1 eec ly5t ', 'low': 129.73, 'lowTime': 1677452154180, 'lowSource': 'n  5eieueri epaydmct1dl', 'latestPrice': 134.6, 'latestSource': 'Close', 'latestTime': 'January 14, 2021', 'latestUpdate': 1611423323421, 'latestVolume': 93379991, 'iexRealtimePrice': 135.06, 'iexRealtimeSize': 1, 'iexLastUpdated': 1640165425467, 'delayedPrice': 134.21, 'delayedPriceTime': 1621864544202, 'oddLotDelayedPrice': 132.91, 'oddLotDelayedPriceTime': 1639198101949, 'extendedPrice': 132, 'extendedChange': 0.19, 'extendedChangePercent': 0.00152, 'extendedPriceTime': 1637279132866, 'previousClose': 135.91, 'previousVolume': 90354066, 'change': -2.05, 'changePercent': -0.0157, 'v

### Um pouco sobre fstrem

Eu uso fstream para modificar alguma tag.

Por exemplo no código abaixo, eu vou trocar a tag {adjective} pela variável adjective acima.

In [7]:
adjective = "superb"
string = f"freeCodeCamp is {adjective}"
print(string)

freeCodeCamp is superb


Parei em 37:45

## Fazer parser do resultado que eu peguei na request da API

In [8]:
price = data['latestPrice']
market_cap = data['marketCap']
print(market_cap)

2201512890845


## Adicionar as informações das ações em um Pandas DataFrame

Para eu passar as informações para um Pandas DataFrame eu preciso antes formatar o DataFrame

In [9]:
# Criar as colunas do dataframe
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


Agora que eu criei o DataFrame, eu vou fazer um append das minhas informações no DataFrame que eu criei logo acima.

Para eu poder adicionar uma informação no meu dataframe eu preciso converter a minha informação em um Panda series. então, se eu tiver um vetor em python que eu quero fazer um append em um dataframe, antes eu preciso converter o vetor em um `pandas series`. 

In [10]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    index = my_columns
    ),
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,134.6,2201512890845,N/A


## Adicionando todas as informações das ações no DataFrame usando loop

Eu tenho uma lista com todos as açções. então eu vou usar a minha lista para passar o nome de cada ação para o endpoint que eu pego as informaçoes da ação na minha API.

O código abaixo vai executar muito demorado. Pois ele vai fazer uma request por vez.
Depois eu vou fazer uma forma que eu faça todos os request em batch de uma única só vez.
Pra não demorar tanto eu vou fazer o append apenas das 5 primeiras ações da minha lista de 500 ações.

In [13]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker'][:5]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
    [
        stock,
        data['latestPrice'],
        data['marketCap'],
        'N/A'
    ],
    index = my_columns),
    ignore_index = True)

In [14]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,129.03,39682083001,N/A
1,AAL,17.19,10274610051,N/A
2,AAP,172.73,11990105808,N/A
3,AAPL,129.62,2214557212029,N/A
4,ABBV,112.90,197916108934,N/A


## Utilizando Batch API Calls para melhorar a performace 

Primeiro eu preciso fazer um split da minha lista em sublistas.

In [16]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [31]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
# symbol_strings vai ser uma lista com todos os nomes de todas as ações
# que eu estou trabalhando. 
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))



[0         A
 1       AAL
 2       AAP
 3      AAPL
 4      ABBV
       ...  
 95     CINF
 96       CL
 97      CLX
 98      CMA
 99    CMCSA
 Name: Ticker, Length: 100, dtype: object,
 100     CME
 101     CMG
 102     CMI
 103     CMS
 104     CNC
        ... 
 195    FTNT
 196     FTV
 197      GD
 198      GE
 199    GILD
 Name: Ticker, Length: 100, dtype: object,
 200     GIS
 201      GL
 202     GLW
 203      GM
 204    GOOG
        ... 
 295     MAA
 296     MAR
 297     MAS
 298     MCD
 299    MCHP
 Name: Ticker, Length: 100, dtype: object,
 300     MCK
 301     MCO
 302    MDLZ
 303     MDT
 304     MET
        ... 
 395     RHI
 396     RJF
 397      RL
 398     RMD
 399     ROK
 Name: Ticker, Length: 100, dtype: object,
 400     ROL
 401     ROP
 402    ROST
 403     RSG
 404     RTX
        ... 
 495    XLNX
 496     XOM
 497    XRAY
 498     XRX
 499     XYL
 Name: Ticker, Length: 100, dtype: object,
 500     YUM
 501     ZBH
 502    ZBRA
 503    ZION
 504     ZTS
 Name

Parei em 1:02:46